In [2]:
import numpy as np
import h5py

from PyQt5.QtWidgets import QFileDialog

from lib.parse_dlc_csv import parse_dlc_csv
from lib.constraints import likelihood_constrain, velocity_constrain, edge_constrain
from lib.plots import plotPerrCDF, plotVelocityCDF, plotRelEdgeLenDistr
from lib.stickman import stickman
from lib.qt_wrapper import gui_fname, gui_fpath
from lib.sampling import selectUniform

### Enter parameters for the analysis of DLC marking

In [2]:
param = {
  "NODE_NAMES"       : ["ShoulderBlade", "Shoulder", "Wrist", "Finger", "Tip"],
  "EDGE_NODES"       : [[0,1], [1,2], [2,3], [3,4]],
  "NODE_MAX_V"       : [70, 70, 70, 70, 70],
  "EDGE_MIN_R"       : [0.5, 0.5, 0.5, 0.5],
  "EDGE_MAX_R"       : [2, 2, 2, 2],
  "LIKELIHOOD_THR"   : 0.05,
  "STICKMAN_CROP_X"  : None,
  "STICKMAN_CROP_Y"  : None
}

param["AVI_FNAME"] = gui_fname("Select original video file...", "./", "Video Files (*.avi)")
param["CSV_FNAME"] = gui_fname("Select tracking file...", "./", "CSV Files (*.csv)")
param["REZ_FPATH"] = gui_fpath("Select result file", "./")

print("Using original video", param["AVI_FNAME"])
print("Using tracking file", param["CSV_FNAME"])
print("Results will be saved in", param["REZ_FPATH"])

### Analyse data, compute and plot results

Determine frames, that
* Have low confidence based on DLC self-reported analysis
* Do not fulfill node constraints (e.g. excessive velocity)
* Do not fulfill edge constraints (e.g. too large or too small edges)

In [ ]:
#########################
# Parse CSV file
#########################

X, Y, P = parse_dlc_csv(fname, param):
nFrames, nNodes = X.shape
    
#########################
# Compute Constraints
#########################
nodeLowConf, perr = likelihood_constrain(P, param)
V, VLowConf, nodeBadV = velocity_constrain(X, Y, nodeLowConf, param)
edgeLength, edgeLowConf, edgeBadLength = edge_constrain(X, Y, nodeLowConf, param)

framesLowConf    = np.sum(nodeLowConf, axis=1) > 0
framesBadV       = np.hstack(([0], np.sum(nodeBadV, axis=1) > 0))
framesBadEdgeLen = np.sum(edgeBadLength, axis=1) > 0
framesBadTotal   = np.logical_or(framesLowConf, framesBadV, framesBadEdgeLen)

print("Average lengths of edges are", edgeLenAvg)
print("Total number of unconfident frames", np.sum(framesLowConf > 0))
print("Total number of frames with bad velocity", np.sum(framesBadV))
print("Total number of frames with bad edges", np.sum(framesBadEdgeLen))
print("Total number of frames bad in any way", np.sum(framesBadTotal))

#########################
# Plot Statistics
#########################
fig, ax = plt.subplots(nrows=2, ncols=2)

plotPerrCDF(ax[0][0], perr, param)                            # Plot CDF of node confidence
plotVelocityCDF(ax[1][0], V, VLowConf, param)                 # Plot CDF of node velocities
plotRelEdgeLenDistr(ax[0][1], edgeLength, edgeLowConf, param) # Plot relative edge length distributions

plt.show()

### Create stickman video and write it to file

In [ ]:
#########################
# Make Stickman Video
#########################

# Use velocity as node constraint for stickman plot.
# There is 1 less velocities than frames - fill with zeros, as we can't estimate velocity for the first frame
nodeLowConstr = np.vstack((np.zeros(nNodes), nodeBadV))

# Use edge length as edge constraint for stickman plot
edgeLowConstr = np.copy(edgeBadLength)

# Write stickman video
stickman(X, Y, param, nodeLowConf, nodeLowConstr, edgeLowConf, edgeLowConstr)

### Decide which frames will be selected for manual marking

1. Check which constraints are present
2. Merge constraints to decide which frames are bad, possibly rank them
3. Extract numbers of frames that are bad
4. Subsample fixed amount randomly, possibly rank-biased
5. Extract actual frames, save as images 

**TODO**:
* Rank-based selection
* Clustering-based selection

In [ ]:
frameIdxs = np.linspace(1, nFrames, nFrames).astype(int)
frameIdxsBad = frameIdxs[framesBadTotal]
nBadFrames = len(frameIdxsBad)

# Option 1: Select all bad frames
# selectedFrames = frameIdxsBad

# Option 2: Select fixed amount of bad frames uniformly
selectedFrames = frameIdxsBad[selectUniform(nBadFrames, 40)]

print("Selected frames are", selectedFrames)

### Extract actual frames from the video, save them to the tracking folder

**TODO**:
* Inspect DLC outlier selection, perhaps can reuse functionality

# Tracking-Only
### Post-tracking marking

**TODO**
1. Run DLC_GUI on all selected frames
2. Open resulting file, merge markings with current markings

### Finalizing

1. Mark all unconfident or unconstrained data as NAN
2. Save data to file

In [ ]:
# Set all bad coordinates to NAN
X[framesBadTotal, :] = np.nan
Y[framesBadTotal, :] = np.nan

rez_fname_h5 = os.path.join(param["REZ_FPATH"], "result.h5")
print("Writing tracking data to", rez_fname_h5)
rezfile = h5py.File(rez_fname_h5, "w")
rezfile['NODE_NAMES'] = param['NODE_NAMES']
rezfile['X'] = X
rezfile['Y'] = Y
rezfile.close()